In [74]:
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import pandas as pd
import numpy as np
import torch.nn.functional as F
from torch.utils.data import random_split

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [58]:
device = torch.device("cuda")
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)


In [7]:
# importing the zipfile module
from zipfile import ZipFile

# loading the temp.zip and creating a zip object
with ZipFile("/content/drive/MyDrive/train.zip", 'r') as zObject:

    # Extracting specific file in the zip
    # into a specific location.
    zObject.extractall(path="/content/unzip")
zObject.close()

In [15]:
# transform = transforms.Compose(
#     [transforms.RandomHorizontalFlip(0.5),
#      transforms.RandomVerticalFlip(0.5),
#      transforms.RandomRotation(10),
#      transforms.ToTensor(),
#      transforms.RandomErasing(0.1),
#      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
# batch_size=64


# data_set = datasets.ImageFolder("/content/unzip/train/", transform = transform)

In [75]:
transform = transforms.Compose(
     [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
batch_size=64


data_set = datasets.ImageFolder("/content/unzip/train/", transform = transform)

In [27]:
data_set[-1]

(tensor([[[-1.0000, -1.0000, -1.0000,  ..., -0.2471, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -0.3647, -1.0000, -1.0000],
          [-0.0353,  0.0039,  0.0431,  ..., -0.3490, -1.0000, -1.0000],
          ...,
          [-1.0000, -1.0000, -0.2706,  ..., -0.7255, -0.6941, -0.7098],
          [-1.0000, -1.0000, -0.3020,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -0.3490,  ..., -1.0000, -1.0000, -1.0000]],
 
         [[-1.0000, -1.0000, -1.0000,  ..., -0.1373, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -0.2549, -1.0000, -1.0000],
          [ 0.0745,  0.1137,  0.1294,  ..., -0.2392, -1.0000, -1.0000],
          ...,
          [-1.0000, -1.0000, -0.1451,  ..., -0.6078, -0.5765, -0.5922],
          [-1.0000, -1.0000, -0.1765,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -0.2235,  ..., -1.0000, -1.0000, -1.0000]],
 
         [[-1.0000, -1.0000, -1.0000,  ..., -0.1137, -1.0000, -1.0000],
          [-1.0000, -1.0000,

In [76]:
dataset_size = len(data_set)

train_size = int(0.8 * dataset_size)
val_size = dataset_size - train_size


train_set, val_set = random_split(data_set, [train_size, val_size])

In [77]:
transform2 = transforms.Compose([
    transforms.RandomHorizontalFlip(0.5),
    transforms.RandomVerticalFlip(0.5),
    transforms.RandomRotation(10),
    transforms.RandomErasing(0.1),
])

train_set.transform = transform2

In [78]:
trainloader = torch.utils.data.DataLoader(train_set, batch_size=batch_size,
                                          shuffle=True, num_workers=8)
valloader = torch.utils.data.DataLoader(val_set, batch_size=batch_size,
                                         shuffle=False, num_workers=8)

In [30]:
#Więcej warstw konwolucyjnych, mniejszy kernel

class Net_More(nn.Module):
    def __init__(self):
        super().__init__()
        ## Warstwa konwolucyjna
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=4, stride=1, padding=0)
        ## Warstwa max pooling
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(64, 256, 4)
        self.pool2 = nn.MaxPool2d(2)
        self.conv3 = nn.Conv2d(256,512,3)
        self.pool3 = nn.MaxPool2d(2,2,1)
        self.conv4 = nn.Conv2d(512,64,3)
        self.fc1 = nn.Linear(1024, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 50)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        # print(x.size())
        x = self.pool2(F.relu(self.conv2(x)))
        # print(x.size())
        x = self.pool3(F.relu(self.conv3(x)))
        # print(x.size())
        x = F.relu(self.conv4(x))
        # print(x.size())
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        # print(x.size())
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [69]:
#Mniej warstw konwolucyjnych, większy kernel, więcej paddingu

class Net_Less(nn.Module):
    def __init__(self):
        super().__init__()
        ## Warstwa konwolucyjna
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=128, kernel_size=5, stride=1, padding=1)
        ## Warstwa max pooling
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(128, 512, 5,1)
        self.pool2 = nn.MaxPool2d(2,2,1)
        self.conv3 = nn.Conv2d(512,128,4)
        self.pool3 = nn.MaxPool2d(2,2)
        self.fc1 = nn.Linear(3200, 500)
        self.fc2 = nn.Linear(500, 100)
        self.fc3 = nn.Linear(100, 50)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        # print(x.size())
        x = self.pool2(F.relu(self.conv2(x)))
        # print(x.size())
        x = self.pool3(F.relu(self.conv3(x)))
        # print(x.size())
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        # print(x.size())
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [70]:
import torch.optim as optim

net = Net_Less().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [71]:
num_epochs = 50

for epoch in range(num_epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()

    print('[%d/%d] loss: %.3f' %
          (epoch+1 ,  num_epochs, running_loss / 2000))
    running_loss = 0.0

print('Finished Training')

[1/50] loss: 1.819
[2/50] loss: 1.594
[3/50] loss: 1.481
[4/50] loss: 1.412
[5/50] loss: 1.362
[6/50] loss: 1.323
[7/50] loss: 1.292
[8/50] loss: 1.266
[9/50] loss: 1.243
[10/50] loss: 1.223
[11/50] loss: 1.208
[12/50] loss: 1.196
[13/50] loss: 1.181
[14/50] loss: 1.165
[15/50] loss: 1.154
[16/50] loss: 1.144
[17/50] loss: 1.132
[18/50] loss: 1.121
[19/50] loss: 1.118
[20/50] loss: 1.106
[21/50] loss: 1.094
[22/50] loss: 1.090
[23/50] loss: 1.084
[24/50] loss: 1.075
[25/50] loss: 1.068
[26/50] loss: 1.064
[27/50] loss: 1.058
[28/50] loss: 1.052
[29/50] loss: 1.049
[30/50] loss: 1.042
[31/50] loss: 1.041
[32/50] loss: 1.030
[33/50] loss: 1.026
[34/50] loss: 1.021
[35/50] loss: 1.018
[36/50] loss: 1.012
[37/50] loss: 1.007
[38/50] loss: 1.006
[39/50] loss: 1.000
[40/50] loss: 0.997
[41/50] loss: 0.997
[42/50] loss: 0.993
[43/50] loss: 0.988
[44/50] loss: 0.982
[45/50] loss: 0.982
[46/50] loss: 0.974
[47/50] loss: 0.973
[48/50] loss: 0.968
[49/50] loss: 0.968
[50/50] loss: 0.958
Finished 

In [79]:
correct = 0
total = 0
with torch.no_grad():
    for data in valloader:
        images, labels = data
        images = images.to(device)
        # calculate outputs by running images through the network
        outputs = net(images).cpu()
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network: %d %%' % (
    100 * correct / total))

Accuracy of the network: 53 %
